In [2]:
import tensorflow as tf
from tensorflow import keras
#import kerastuner as kt 

ModuleNotFoundError: ignored

In [3]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 71kB 5.6MB/s 


In [4]:
import kerastuner as kt

In [5]:
(trainx, trainy),(testx,testy) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [6]:
trainx = trainx/255.0
testx = testx/255.0

In [7]:
len(trainx[0])

28

In [13]:
layer1 = keras.layers.Flatten(input_shape=(28,28))
layer2 = keras.layers.Dense(16, activation='relu')
layer3 = keras.layers.Dense(32, activation='relu')
layer4 = keras.layers.Dense(10, activation='softmax')


In [47]:

def hypermodel(hps):
  hp_ss = kt.HyperParameters()

  hp_hiddenunits_1 = hp_ss.Int('units', min_value=32, max_value=512, step=32)
  layer1 = keras.layers.Flatten(input_shape=(28,28))
  layer2 = keras.layers.Dense(hp_hiddenunits_1, activation='relu')
  layer3 = keras.layers.Dense(32, activation='relu')
  layer4 = keras.layers.Dense(10, activation='softmax')
  model = keras.Sequential([layer1,layer2,layer3,layer4])
  hp_lr = hp_ss.Choice('learning_rate', values=[1e-2, 1e-3, 2e-3])
  myopt = keras.optimizers.Adam(learning_rate=hp_lr)
  model.compile(optimizer=myopt, loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
  return model

In [48]:
# Hyperband -> model tuner
#hypermodel-> a method to build and return a model

tuner = kt.Hyperband(hypermodel,
                     objective='val_accuracy',
                     max_epochs=10,
                     directory='tomato')

In [53]:
# early stop search as well-> best no. of epochs the model should train to
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=5)

In [54]:
tuner.search(trainx, trainy, epochs=50)

INFO:tensorflow:Oracle triggered exit


In [51]:
besthp = tuner.get_best_hyperparameters(num_trials=1)
besthp

[]